In [29]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from datetime import datetime, date
import time
import pygrib
import pandas as pd
from dateutil.relativedelta import relativedelta
import os

last_report_fn = os.listdir('./reports')[0]
ct_df = pd.read_csv("usregions.csv")[:30000]
lls = list(zip(ct_df['lat'], ct_df['lng']))
last_report = pd.read_csv(f"./reports/{last_report_fn}").to_dict()
report_dict = {
        "Date": [(date.today()+relativedelta(days=i)).strftime("%a, %d %b %Y") for i in range(16)],
        "Current FC": [],
        "FC 6 hours ago": last_report['Current FC'],
        "FC 12 hours ago": last_report['FC 6 hours ago'],
        "FC 18 hours ago": last_report['FC 12 hours ago'],
        "FC 24 hours ago": last_report['FC 18 hours ago'],
        "Diff 12 hours ago": [],
        "Diff 24 hours ago": []
    }
dt = datetime.now()
file_dt = "".join(list(map(str,[dt.year,dt.month,f"{dt.day:02d}"])))

client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket = client.list_objects(Bucket='noaa-gefs-pds', Prefix = "gefs."+file_dt+"/", Delimiter="/",)

while True:
    if bucket.get('CommonPrefixes') is not None:
        fldr = bucket.get('CommonPrefixes')[-1].get('Prefix')
        break
    else:
        time.sleep(300)

cycle = fldr.split('/')[-2]
base = f'{fldr[:-1]}/atmos/pgrb2ap5/gec00.t{cycle}z.pgrb2a.0p50.f'
files = []
for i in range(0,243,3):
    files.append(base+f"{i:03d}")
for i in range(246,390,6):
    files.append(base+f"{i}")


def process(file, cycle):
    grbs = pygrib.open(file)
    grb = grbs.select(name='Temperature')[-1]
    data = grb.data()[0]
    
    def f(lat, lon):
        return round(abs(data[int(round(-2*lat)+180),int(round(2*lon))]*1.8-459.67-65),2)
    
    ct_df['dd_'+cycle] = [f(lat, lon) for lat,lon in lls]
    
    grbs.close()
def _to_state(x):
    return pd.Series({'dd':(x['population'] * x['dd']/x['population'].sum()).sum(),'population': x['population'].sum()})
def _to_region(x):
    return pd.Series({'dd':(x['population'] * x['dd']/x['population'].sum()).sum(),'population': x['population'].sum()})
while files:
    idx = int(files[0].split(".")[-1][-3:])

    print(files[0])
    try:
        if idx%24==0 and idx>0:
            ct_df['dd'] = (ct_df['dd_00']+ct_df['dd_03']+ct_df['dd_06']+ct_df['dd_09']+ct_df['dd_12']+ct_df['dd_15']+ct_df['dd_18']+ct_df['dd_21'])/8
            state_df = ct_df.groupby(['state_name','region'])[['population', 'dd']].apply(_to_state).reset_index()
            region_df = state_df.groupby(['region'])[['population','dd']].apply(_to_region).reset_index()
            national_dd = round((region_df['population']*region_df['dd']).sum()/region_df['population'].sum(),2)
            report_dict['Current FC'].append(national_dd)
            
            report_df = pd.DataFrame(dict([ (k,pd.Series(v,dtype=pd.Float32Dtype)) for k,v in report_dict.items() ]))
            report_df['Diff 12 hours ago'] = report_df['Current FC']-report_df['FC 12 hours ago']
            report_df['Diff 24 hours ago'] = report_df['Current FC']-report_df['FC 24 hours ago']
            report_df['Diff 12 hours ago'].round(2)
            report_df['Diff 24 hours ago'].round(2)
            report_df.to_csv(f'reports/report_{str(date.today())}_{cycle}', index=False)
        client.download_file('noaa-gefs-pds', files[0], 'temp')
        process('temp', f"{idx%24:02d}")
        
    except Exception as e:
        print(e)
        # time.sleep(60)
        break
    del files[0]

    


gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f000
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f003
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f006
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f009
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f012
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f015
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f018
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f021
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f024


C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\15312

gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f027
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f030
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f033
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f036
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f039
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f042
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f045
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f048


C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\15312

gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f051
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f054
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f057
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f060
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f063
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f066
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f069
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f072


...

C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\15312

gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f342
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f348
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f354
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f360


C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\15312

gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f366
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f372
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f378
gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f384


C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\1531283826.py:72: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
C:\Users\anton\AppData\Local\Temp\ipykernel_38096\15312

['.git',
 'front.py',
 'report.csv',
 'report_2023-11-11_06',
 'temp',
 'test.ipynb',
 'uscities.csv',
 'usregions.csv']

In [21]:
ct_df = pd.read_csv("uscities.csv")
states_to_regions = {"Washington": "Pacific", "Oregon":"Pacific","California":"Pacific","Nevada":"Mountain","Wyoming":"Mountain","Arizona":"Mountain","Montana":"Mountain","Idaho":"Mountain","Utah":"Mountain","Colorado":"Mountain","New Mexico":"Mountain","North Dakota":"Midwest","South Dakota":"Midwest","Nebraska":"Midwest","Kansas":"Midwest","Minnesota":"Midwest","Iowa":"Midwest","Missouri":"Midwest","Wisconsin":"Midwest","Michigan":"Midwest","Illinois":"Midwest","Indiana":"Midwest","Ohio":"Midwest","New York":"Northeast","Pennsylvania":"Northeast","New Jersey":"Northeast","Vermont":"Northeast","New Hampshire":"Northeast","Maine":"Northeast","Massachusetts":"Northeast","Rhode Island":"Northeast","Connecticut":"Northeast","Texas":"South","Oklahoma":"South","Arkansas":"South","Louisiana":"South","North Carolina":"South","South Carolina":"South","Florida":"South","Georgia":"South","West Virginia":"South","Virginia":"South","Delaware":"South","Maryland":"South","Tennessee":"South","Mississippi":"South","Alabama":"South","Kentucky":"South","District of Columbia":"South","Puerto Rico":"lol","Hawaii":"lol","Alaska":"lol"}


In [22]:
ct_df["region"] =[states_to_regions[x] for x in ct_df["state_name"]]

ct_df.drop(ct_df[ct_df["region"]=="lol"].index, inplace=True)

In [23]:
ct_df.to_csv("usregions.csv")

In [10]:
df = pd.read_csv('report.csv')
df

,Unnamed: 0,Date,Current FC,FC 6 hours ago,FC 12 hours ago,FC 18 hours ago,FC 24 hours ago,Diff 12 hours ago,Diff 24 hours ago


In [12]:
import pandas as pd
from datetime import datetime, timedelta
import random
import time

# Function to generate a random DataFrame row
def generate_random_row():
    current_time = datetime.now()
    random_values = [random.randint(1, 100) for _ in range(7)]  # Assuming 7 columns with random integer values
    return pd.DataFrame([{
        "Unnamed: 0": None,
        "Date": current_time.strftime("%Y-%m-%d %H:%M:%S"),
        "Current FC": random_values[0],
        "FC 6 hours ago": random_values[1],
        "FC 12 hours ago": random_values[2],
        "FC 18 hours ago": random_values[3],
        "FC 24 hours ago": random_values[4],
        "Diff 12 hours ago": random_values[5],
        "Diff 24 hours ago": random_values[6]
    }])

# Path to the CSV file
csv_file_path = 'report.csv'

# Number of iterations (updates)
num_iterations = 10  # Change this number as needed

for i in range(num_iterations):
    # Read the existing DataFrame from the CSV file
    df = pd.read_csv(csv_file_path)

    # Generate a random row and append it to the DataFrame
    new_row = generate_random_row()
    df = pd.concat([df, new_row], ignore_index=True)

    # Save the updated DataFrame back to the CSV file
    df.to_csv(csv_file_path, index=False)

    print(f"Updated CSV file. Iteration {i + 1} / {num_iterations}")

    # Wait for 1 minute before the next update
    time.sleep(30)


Updated CSV file. Iteration 1 / 10
Updated CSV file. Iteration 2 / 10
Updated CSV file. Iteration 3 / 10
Updated CSV file. Iteration 4 / 10
Updated CSV file. Iteration 5 / 10
Updated CSV file. Iteration 6 / 10


KeyboardInterrupt: 